In [41]:
import geopandas as gpd
import folium as fm
import pandas as pd
import param
import panel as pn
import random
pn.extension(sizing_mode="stretch_width")

### Summary Statistics using Maps

In [42]:


# read segment shapefile into spatial enabled dataframe
dirSegShp = r'A:\1 - TDM\1 - Input Dev\6 - Segment\0 - Segment Shapefile\Master_Segs_withFactors_20220915\Master_Segs_withFactors_20220915shp.shp'
segShp = gpd.read_file(dirSegShp)
segShp.head()


,Id,SEGID,BMP,EMP,DISTANCE,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,FC_NAME,...,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,F2050,F2040,FORNOTES,ForecastSo,Shape_Leng,geometry
0,0,0006_000.0,0.000,0.665,0.666641,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,1.1065,0,0.2496,0.2324,600,550,None,Utah_Statewide_Traffic_Volume_Historic_and_For...,1072.856212,"LINESTRING (236177.700 4327541.250, 237241.180..."
1,0,0006_000.7,0.665,16.022,15.369839,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,1.1065,0,0.2496,0.2324,550,500,None,Utah_Statewide_Traffic_Volume_Historic_and_For...,24735.407823,"LINESTRING (237241.180 4327399.720, 238150.600..."
2,0,0006_016.0,16.022,46.017,30.001961,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,1.1065,0,0.2496,0.2324,550,500,None,Utah_Statewide_Traffic_Volume_Historic_and_For...,48283.572122,"LINESTRING (261403.900 4327045.400, 261409.000..."
3,0,0006_046.0,46.017,60.218,14.194306,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,1.1065,0,0.1751,0.3338,500,450,None,Utah_Statewide_Traffic_Volume_Historic_and_For...,22843.567118,"LINESTRING (305380.160 4325741.850, 305784.500..."
4,0,0006_060.2,60.218,77.545,17.323237,HWY 6,Arterial,3,Arterial,Other Principal Arterial,...,1.1065,0,0.1751,0.3338,500,450,None,Utah_Statewide_Traffic_Volume_Historic_and_For...,27879.103631,"LINESTRING (324618.000 4337936.900, 324647.400..."


In [43]:
segShp.to_file('segShp.geojson', driver='GeoJSON')
segShp = gpd.read_file('segShp.geojson')

In [63]:
m = fm.Map(
    location=[40.7608, -111.8910],
    tiles="cartodbpositron",
    zoom_start=9,
)



def style_function(feature):
    volume = employed_series.get(int(feature["id"][-5:]), None)
    return {
        "fillOpacity": 0.5,
        "weight": 0,
        "fillColor": "#black" if employed is None else colorscale(employed),
    }




fm.GeoJson(segShp, name="geojson",
  style_function= lambda x :{'color':'blue' if x['properties']['F2050']>10000 \
       else 'orange' if 100000 <x['properties']['F2050']>1000000 else 'red'},

m

SyntaxError: keyword argument repeated: style_function (1650921837.py, line 6)

In [36]:
#dirSegShp = r'A:\1 - TDM\1 - Input Dev\6 - Segment\0 - Segment Shapefile\Master_Segs_withFactors_20220915\Master_Segs_withFactors_20220915shp.shp'

#import into spatially-enabled DataFrame
#sdf_Forecasts = pd.DataFrame.spatial.from_featureclass(dirSegShp)
#sdf_Forecasts_short = sdf_Forecasts[['SEGID','SHAPE']]

In [ ]:
#Define coordinates of where we want to center our map
boulder_coords = [40.015, -105.2705]

map_with_tiles = fm.Map(location = boulder_coords, tiles = 'cartodb positron')
map_with_tiles
#folium.PolyLine(segShp['geometry']).add_to(map_with_tiles)

In [37]:
def get_map(lat=40.7608, long=-111.8910, zoom_start=10):
    return fm.Map(location=[lat,long], zoom_start=zoom_start)

map = get_map()

pn.panel(map, height=400)

BokehModel(combine_events=True, render_bundle={'docs_json': {'de212126-6f2c-4173-a0fe-31b210411144': {'defs': …

In [30]:
def get_df_aqi(lat_bounds=(39.7,40.7), long_bounds=(-112.8,-110.8), points_count=40):
    aqi = {
        "Latitude": [random.uniform(*lat_bounds) for _ in range(0,points_count)],
        "Longitude": [random.uniform(*long_bounds) for _ in range(0,points_count)],
        "AQI": [random.uniform(0,500) for _ in range(0,points_count)],
    }
    return pd.DataFrame(aqi)

df_aqi = get_df_aqi()
df_aqi.sample(5)

,Latitude,Longitude,AQI
28,40.148579,-112.197780,183.952573
29,40.132458,-111.659053,78.345260
17,39.728217,-112.351993,190.916952
31,40.007493,-112.156843,354.054648
20,39.927185,-111.225625,422.836726


In [39]:
def add_aqi_circles(map, df_aqi):
    green_p1  = fm.map.FeatureGroup()
    yellow_p1 = fm.map.FeatureGroup()
    orange_p1 = fm.map.FeatureGroup()
    red_p1    = fm.map.FeatureGroup()
    purple_p1 = fm.map.FeatureGroup()
    maroon_p1 = fm.map.FeatureGroup()

    for _, row in df_aqi.iterrows():
        if row.AQI<50:
            feature_group = green_p1
            fill_color = "green"
        elif row.AQI < 100:
            feature_group = yellow_p1
            fill_color = "yellow"
        elif row.AQI < 150:
            feature_group = orange_p1
            fill_color = "orange"
        elif row.AQI < 200:
            feature_group = red_p1
            fill_color = "red"
        elif row.AQI < 300:
            feature_group = purple_p1
            fill_color='purple'
        else:
            feature_group = maroon_p1
            fill_color = "maroon"

        feature_group.add_child(
            fm.CircleMarker(
                [row.Latitude, row.Longitude],
                radius=10, 
                fill=True,
                fill_color=fill_color,
                fill_opacity=0.7
            )
        )

    map.add_child(green_p1)
    map.add_child(yellow_p1)
    map.add_child(orange_p1)
    map.add_child(red_p1)
    map.add_child(purple_p1)

add_aqi_circles(map, df_aqi)
fm.GeoJson(os.path.join('segShp.geojson')).add_to(map)
pn.panel(map, height=400)

BokehModel(combine_events=True, render_bundle={'docs_json': {'057176d1-2218-4e4c-9129-7035e3cd7906': {'defs': …

In [33]:
class PanelFoliumMap(param.Parameterized):
    points_count = param.Integer(20, bounds=(10,100))
        
    def __init__(self, **params):
        super().__init__(**params)
        self.map = get_map()
        self.folium_pane = pn.pane.plot.Folium(sizing_mode="stretch_both", min_height=500, margin=0)    
        self.view = pn.Column(
            self.param.points_count,
            self.folium_pane,
            sizing_mode="stretch_both", height=500
        )
        self._update_map()

    @param.depends("points_count", watch=True)
    def _update_map(self):
        self.map = get_map()
        df_aqi = get_df_aqi(points_count=self.points_count)
        add_aqi_circles(self.map, df_aqi)
        self.folium_pane.object = self.map

        
app = PanelFoliumMap()
app.view

BokehModel(combine_events=True, render_bundle={'docs_json': {'d8c5902e-e93d-4f07-a1c0-46471ca51d76': {'defs': …